# Air quality project

## Project and data description

### Project

czy potrzeba stacji więcej/ mniej. 

zrobić wizualizacje/ mapkę czy są zagęszone

Jak zwiększenie w konkretnych miejsacach wpłynie na zanieczyszczenie w pozostalych? 

jaka gęstość czujników? 

Jaka jest rzeźba terenu? Kraków jest w niecce. 

### Data

The dataset consists air quality data (the concentrations of particulate matter PM1, PM2.5 and PM10, temperature, air pressure and humidity) from 2017 generated by network of 56 low-cost sensors located in Krakow, Poland.

### The goal

## Preprocessing and analysis

In [18]:
import glob
import os
import pandas as pd

In [4]:
# the name of the data frame corresponds to month ex. 01 is january. 
path = r'C:\Users\User\PycharmProjects\Air_quality\Data_files\january-2017.csv'

df_01 = pd.read_csv(path)

In [5]:
df_01.head()

,UTC time,3_temperature,3_humidity,3_pressure,3_pm1,3_pm25,3_pm10,140_temperature,140_humidity,140_pressure,...,857_pressure,857_pm1,857_pm25,857_pm10,895_temperature,895_humidity,895_pressure,895_pm1,895_pm25,895_pm10
0,2017-01-01T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01T01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01T02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-01T03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01T04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_01.shape

(744, 337)

In [8]:
df_01.columns

Index(['UTC time', '3_temperature', '3_humidity', '3_pressure', '3_pm1',
       '3_pm25', '3_pm10', '140_temperature', '140_humidity', '140_pressure',
       ...
       '857_pressure', '857_pm1', '857_pm25', '857_pm10', '895_temperature',
       '895_humidity', '895_pressure', '895_pm1', '895_pm25', '895_pm10'],
      dtype='object', length=337)

In [11]:
df_01.info

<bound method DataFrame.info of                 UTC time  3_temperature  3_humidity  3_pressure  3_pm1  \
0    2017-01-01T00:00:00            NaN         NaN         NaN    NaN   
1    2017-01-01T01:00:00            NaN         NaN         NaN    NaN   
2    2017-01-01T02:00:00            NaN         NaN         NaN    NaN   
3    2017-01-01T03:00:00            NaN         NaN         NaN    NaN   
4    2017-01-01T04:00:00            NaN         NaN         NaN    NaN   
..                   ...            ...         ...         ...    ...   
739  2017-01-31T19:00:00            NaN         NaN    101783.0   82.0   
740  2017-01-31T20:00:00            NaN         NaN    101813.0   95.0   
741  2017-01-31T21:00:00            NaN         NaN    101835.0  106.0   
742  2017-01-31T22:00:00            NaN         NaN    101866.0   99.0   
743  2017-01-31T23:00:00            NaN         NaN    101887.0   99.0   

     3_pm25  3_pm10  140_temperature  140_humidity  140_pressure  ...  \
0     

In [14]:
def number_of_Nans(data_frame):
    print('Liczba brakujących wartości w poszczególnych kolumnach:')
    column_names = data_frame.columns
    for column in column_names:
        print(column,': ', data_frame[column].isnull().sum())

In [17]:
56*6

336

In [15]:
number_of_Nans(df_01)

Liczba brakujących wartości w poszczególnych kolumnach:
UTC time :  0
3_temperature :  726
3_humidity :  726
3_pressure :  409
3_pm1 :  409
3_pm25 :  409
3_pm10 :  409
140_temperature :  744
140_humidity :  744
140_pressure :  744
140_pm1 :  744
140_pm25 :  744
140_pm10 :  744
142_temperature :  53
142_humidity :  53
142_pressure :  53
142_pm1 :  78
142_pm25 :  78
142_pm10 :  78
147_temperature :  1
147_humidity :  1
147_pressure :  1
147_pm1 :  1
147_pm25 :  1
147_pm10 :  1
169_temperature :  112
169_humidity :  112
169_pressure :  112
169_pm1 :  112
169_pm25 :  112
169_pm10 :  112
170_temperature :  0
170_humidity :  0
170_pressure :  0
170_pm1 :  0
170_pm25 :  0
170_pm10 :  0
171_temperature :  1
171_humidity :  1
171_pressure :  1
171_pm1 :  1
171_pm25 :  1
171_pm10 :  1
172_temperature :  744
172_humidity :  744
172_pressure :  1
172_pm1 :  1
172_pm25 :  1
172_pm10 :  1
173_temperature :  127
173_humidity :  127
173_pressure :  0
173_pm1 :  0
173_pm25 :  0
173_pm10 :  0
174_temper

In [22]:
root_path = r'C:\Users\User\PycharmProjects\Air_quality\Data_files'
all_files = glob.glob(os.path.join(root_path,'*.csv'))
all_files

['C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\april-2017.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\august-2017.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\december-2017.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\february-2017.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\january-2017.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\july-2017.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\june-2017.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\march-2017.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\may-2017.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\november-2017.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\october-2017.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\\Data_files\\sensor_locations.csv',
 'C:\\Users\\User\\PycharmProjects\\Air_quality\

In [ ]:
df_ = read_csv()
    
    